# YouTube 자막 번역기: 영어에서 한국어로

이 노트북은 YouTube 동영상의 영어 자막을 한국어로 번역하는 도구입니다. Anthropic의 Claude AI를 사용하여 고품질의 번역을 제공합니다.

## 설정

먼저 필요한 라이브러리를 설치하고 필수 모듈을 가져옵니다.

In [ ]:
# 필요한 라이브러리 설치
!pip install requests
!pip install anthropic

In [3]:
# 필요한 모듈 import
import os
import sys
import anthropic
from typing import List
from google.colab import userdata, drive, files

In [ ]:
# Google Drive 마운트
drive.mount('/content/drive')

In [5]:
# 상수 정의
BATCH_SIZE = 5  # 한 번에 처리할 자막의 수

INPUT_TOKEN_COST = 3 / 1_000_000  # 1M 토큰당 $3
OUTPUT_TOKEN_COST = 3.75 / 1_000_000  # 1M 토큰당 $3.75

## 유틸리티 함수

다음은 프로그램에서 사용되는 여러 유틸리티 함수들입니다.

In [6]:
def get_api_key():
    """
    Google Colab의 사용자 데이터에서 Claude API 키를 가져옵니다.
    키가 없으면 오류 메시지를 출력하고 프로그램을 종료합니다.
    """
    try:
        return userdata.get('CLAUDE_API_KEY')
    except Exception as e:
        print(f"API 키 접근 오류: {e}")
        print("Google Colab에서 'ANTHROPIC_API_KEY' 시크릿을 설정했는지 확인해주세요.")
        sys.exit(1)

def read_srt_file(file_path: str) -> str:
    """
    지정된 경로에서 SRT 파일을 읽어 내용을 문자열로 반환합니다.
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def write_srt_file(file_path: str, content: str):
    """
    주어진 내용을 SRT 파일로 저장합니다.
    """
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(content)

def split_subtitles(srt_content: str) -> List[str]:
    """
    SRT 내용을 개별 자막으로 분할합니다.
    """
    return srt_content.strip().split('\n\n')

def create_batches(subtitles: List[str], batch_size: int) -> List[str]:
    """
    자막 목록을 지정된 크기의 배치로 나눕니다.
    """
    batches = []
    for i in range(0, len(subtitles), batch_size):
        batch = subtitles[i:i+batch_size]
        context_before = get_context_subtitles(subtitles, i, -2)
        context_after = get_context_subtitles(subtitles, i+batch_size, 2)
        batches.append('\n\n'.join(context_before + batch + context_after))
    return batches

def get_context_subtitles(subtitles: List[str], index: int, range: int) -> List[str]:
    """
    주어진 인덱스 주변의 자막을 가져옵니다.
    """
    start = max(0, index + range) if range < 0 else index
    end = min(len(subtitles), index + range) if range > 0 else index
    return subtitles[start:end]


## 번역 함수

다음은 실제 번역 작업을 수행하는 핵심 함수들입니다.

In [7]:
def translate_batch(client: anthropic.Anthropic, batch: str, start_number: int) -> str:
    """
    주어진 배치의 자막을 번역합니다.
    Claude AI에 전송할 시스템 프롬프트를 생성하고, 응답을 처리합니다.
    """
    system_prompt = f"""You are tasked with translating English subtitles to Korean for a YouTube video. Your goal is to create high-quality Korean translations that are natural, error-free, and properly formatted. The subtitles are provided in SRT format. Keep the subtitle length appropriate for easy reading.

Here are the English subtitles you need to translate:

<english_subtitles>
{{ENGLISH_SUBTITLES}}
</english_subtitles>

Follow these instructions to complete the translation:

1. Translate each subtitle from English to Korean, ensuring that the Korean translation is natural and easily understood by native speakers.

2. Keep the subtitle length appropriate. Aim for 40 characters or less per subtitle. If a subtitle becomes too long, split it into two or more subtitles and adjust the timing accordingly. Ensure that the numbering remains sequential.

3. Use appropriate honorifics and formality levels based on the context of the video.

4. If there are any culturally specific references or idioms, adapt them to Korean equivalents that convey the same meaning.

5. For terms that are better left in English (e.g., brand names, technical terms), use the original English term followed by a brief Korean explanation in parentheses if necessary.

6. Use line breaks to make the subtitles easier to read according to the meaning of the content.

7. Consider the context of adjacent subtitles to ensure coherence and natural flow.

Provide your translation in the following format:

<korean_subtitles>
1
[start time] --> [end time]
[Korean translation]

2
[start time] --> [end time]
[Korean translation]

...
</korean_subtitles>

Here's an example of a good translation:

<example>
English:
1
00:00:00,560 --> 00:00:01,570
That's a good idea.

2
00:00:02,630 --> 00:00:03,582
Let's implement it right away.

3
00:00:16,550 --> 00:00:52,826
And basically mark Thomas is, there's no such thing as a lead with Apache open source projects, but he's certainly one of the most continuous and valuable committers to the project and he works on the spring team, but he's also one of the most important people in the Apache Tomcat project and he has been for more than a decade, almost 20 years, and he wrote a blog last year looking at what that means, what virtual threads mean for something like Tomcat, the conclusion is kind of interesting.

4
00:00:52,938 --> 00:00:55,626
He uses some very weird language, but it means the same thing.

Korean:
1
00:00:00,560 --> 00:00:01,570
좋은 생각이네요.

2
00:00:02,630 --> 00:00:03,582
바로 실행해 봅시다.

3
00:00:16,550 --> 00:00:21,326
기본적으로 마크 토마스는, Apache 오픈 소스 프로젝트에는 리더라는 것이 없지만,

4
00:00:21,550 --> 00:00:25,426
그는 확실히 프로젝트에 가장 지속적이고 가치 있는 컨트리뷰터 중 한 명입니다.

5
00:00:25,527 --> 00:00:30,625
그는 스프링 팀에서 일하지만,

6
00:00:30,727 --> 00:00:33,125
또한 Apache Tomcat 프로젝트에서 가장 중요한 인물 중 한 명이며,

7
00:00:33,727 --> 00:00:36,125
10년 이상, 거의 20년 동안 그래왔습니다.

8
00:00:36,228 --> 00:00:43,846
그는 작년에 블로그를 작성했는데,

9
00:00:43,928 --> 00:00:52,826
Tomcat과 같은 프로젝트에 가상 스레드가 어떤 의미를 갖는지에 대해 살펴봤습니다.

그 결론은 꽤 흥미롭습니다.

6
00:00:52,938 --> 00:00:55,626
그는 약간 이상한 표현을 사용하지만, 의미하는 바는 같습니다.
</example>

If you encounter any issues or special cases, handle them as follows:

- For untranslatable proper nouns or technical terms, keep them in English and add a brief Korean explanation in parentheses if necessary.
- If a subtitle contains multiple sentences, try to keep them together if possible. Only split them if the resulting subtitle would be too long.
- If you're unsure about the meaning of a phrase or sentence, provide the best possible translation based on the context.

Remember to review your translation for accuracy, naturalness, and proper formatting before submitting your final output. Ensure that the Korean translation maintains the original meaning while being easily readable and natural-sounding to Korean viewers."""

    try:
        message = client.messages.create(
            model="claude-3-5-sonnet-20240620",
            max_tokens=8000,
            system=system_prompt,
            messages=[
                {"role": "user", "content": batch}
            ]
        )

        # 토큰 사용량 추출
        usage = message.usage
        input_tokens = usage.input_tokens
        output_tokens = usage.output_tokens

        translated_text = message.content[0].text
        korean_subtitles = translated_text.split('<korean_subtitles>')[1].split('</korean_subtitles>')[0].strip()
        return korean_subtitles + '\n\n', input_tokens, output_tokens
    except Exception as e:
        print(f"번역 중 오류 발생: {e}")
        raise

def renumber_subtitles(srt: str) -> str:
    """
    번역된 자막의 번호를 1부터 순차적으로 다시 매깁니다.
    잘못된 번호 또는 빈 줄을 건너뛰고, 올바른 번호로 재정렬합니다.
    """
    subtitles = srt.strip().split('\n\n')
    new_subtitles = []
    counter = 1

    for subtitle in subtitles:
        lines = subtitle.strip().split('\n')
        if len(lines) > 1:
            # 첫 번째 줄을 자막 번호로 대체
            lines[0] = str(counter)
            counter += 1
            new_subtitles.append('\n'.join(lines))

    return '\n\n'.join(new_subtitles)


def translate_subtitles(client: anthropic.Anthropic, input_file: str, output_file: str):
    """
    전체 자막 번역 프로세스를 관리합니다.
    파일을 읽고, 배치로 나누어 번역하고, 결과를 저장합니다.
    """
    srt_content = read_srt_file(input_file)
    subtitles = split_subtitles(srt_content)
    batches = create_batches(subtitles, BATCH_SIZE)

    translated_srt = ""
    subtitle_number = 1

    # 토큰 사용량을 추적하는 변수들
    total_input_tokens = 0
    total_output_tokens = 0

    for i, batch in enumerate(batches, 1):
        print(f"번역 진행 중... {i}/{len(batches)}")
        try:
            translated_batch, input_tokens, output_tokens = translate_batch(client, batch, subtitle_number)
            translated_srt += translated_batch
            subtitle_number += BATCH_SIZE

            # 토큰 사용량 누적
            total_input_tokens += input_tokens
            total_output_tokens += output_tokens

        except Exception as e:
            print(f"배치 {i} 번역 중 오류 발생: {e}")
            print("다음 배치로 진행합니다.")
            continue

    # 자막 번호 재정렬
    translated_srt = renumber_subtitles(translated_srt)
    write_srt_file(output_file, translated_srt)
    print(f"번역 완료! 결과가 {output_file}에 저장되었습니다.")

    # 총 요금 계산
    total_cost = (total_input_tokens * INPUT_TOKEN_COST) + (total_output_tokens * OUTPUT_TOKEN_COST)
    print(f"총 사용된 입력 토큰: {total_input_tokens}")
    print(f"총 사용된 출력 토큰: {total_output_tokens}")
    print(f"총 요금: ${total_cost:.4f}")


## 메인 실행 부분

마지막으로, 프로그램의 주요 실행 부분입니다. API 키를 확인하고, 사용자로부터 입력 파일을 받아 번역을 수행하고 결과를 저장합니다.

In [ ]:
if __name__ == "__main__":
    # API 키 확인 및 클라이언트 생성
    api_key = get_api_key()
    client = anthropic.Anthropic(api_key=api_key)
    print("API 키가 성공적으로 로드되었습니다.")

    # 사용자로부터 입력 파일 업로드 받기
    print("번역할 SRT 파일을 업로드해주세요.")
    uploaded = files.upload()
    input_file_name = list(uploaded.keys())[0]
    input_file_path = f'/content/{input_file_name}'
    print(f"파일 '{input_file_name}'이(가) 업로드되었습니다.")

    # 출력 파일 이름 생성
    base, ext = os.path.splitext(input_file_name)
    output_file_name = f"{base}_ko{ext}"
    output_file_path = f'/content/{output_file_name}'

    # 번역 실행
    try:
        translate_subtitles(client, input_file_path, output_file_path)

        # 번역된 파일 다운로드
        files.download(output_file_path)

        # 번역된 파일을 Google Drive에 저장 (선택사항)
        drive_output_path = f'/content/drive/My Drive/{output_file_name}'
        !cp "{output_file_path}" "{drive_output_path}"
        print(f"번역된 파일이 Google Drive에 저장되었습니다: {drive_output_path}")

    except Exception as e:
        print(f"번역 중 오류가 발생했습니다: {e}")
        print("문제가 지속되면 API 키를 확인하고 Anthropic 지원팀에 문의하세요.")

이제 노트북을 실행할 준비가 되었습니다. 각 셀을 순서대로 실행하면 YouTube 자막을 영어에서 한국어로 번역할 수 있습니다. 번역된 파일은 자동으로 다운로드되며, Google Drive에도 저장됩니다.